In [1]:
#import
import pandas as pd
from fuzzywuzzy import fuzz,process
import numpy as np
import re
pd.set_option('display.max_rows', None)

### DATA KHUSUS BEARING

NOTE
1. kata yang mengandung ukuran seperti MM
2. kata yang mengandung huruf dan angka dipisahkan oleh spasi
3. kata yang mengandung dash atau garis miring
4. kata yang memiliki string panjang
5. kata harus mengandung XBEARING atau BALL BEARING

In [2]:
df = pd.read_excel('C:/Users/Corporate/Documents/ovi/RAW DATA TERBARU/bearing_100323_v1.xlsx')
# df2 = df.drop(['new_kata2','note'],axis=1)
df.head()

,Unnamed: 0,kode_item,nama_item,part_number,jenis_item,kategori_item,status,updated_at,sbu,item_id,...,REF.part_number,REF.coarse_spec,REF.jenis_item,REF.kategori_item,REF.status,REF.sub_cluster,_score,agg_cluster_label_after,agg_cluster_label_REF.nama_item,ref_nama_item
0,0,SPT.P1.DA.00000327,BEARING NTN FLW685ZZA,BEARING NTN FLW685ZZA,Stock,SPAREPART,Active,2021-07-08 03:58:15,IPG,IPG-SPT.P1.DA.00000327,...,NaN,FLW685ZZA,Stock,SPAREPART,Active,BEARING,100.0,0,0.0,BEARING FLW685ZZA
1,1,SPT.P1.BC.00000061,NTN BEARING FLW687ZZ,NTN BEARING FLW687ZZ,Stock,SPAREPART,Active,2019-01-11 09:14:55,IPG,IPG-SPT.P1.BC.00000061,...,NaN,FLW687ZZ,Stock,SPAREPART,Active,BEARING,100.0,0,0.0,BEARING FLW687ZZ
2,2,SPT.P5.MT.00000393,BEARING FYH UCP208J,BEARING FYH UCP208J,Stock,SPAREPART,Active,2021-03-18 01:30:34,IPG,IPG-SPT.P5.MT.00000393,...,NaN,UCP208J,Stock,SPAREPART,Active,BEARING,100.0,7,1.0,BEARING UCP 208 J
3,3,MS01132,BEARING UCP208,NaN,Stock,SPAREPART,Active,2015-01-17 10:16:46,ISP,ISP-MS01132,...,NaN,UCP208,Stock,SPAREPART,Active,BEARING,100.0,7,1.0,BEARING UCP 208
4,4,MS01231,BEARING UC208,NaN,Stock,SPAREPART,Active,2023-09-07 14:38:53.990000,ISP,ISP-MS01231,...,NaN,UC208,Stock,SPAREPART,Active,BEARING,100.0,7,1.0,BEARING UC 208


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1287 entries, 0 to 1286
Data columns (total 28 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       1287 non-null   int64  
 1   kode_item                        1287 non-null   object 
 2   nama_item                        1287 non-null   object 
 3   part_number                      581 non-null    object 
 4   jenis_item                       1287 non-null   object 
 5   kategori_item                    1287 non-null   object 
 6   status                           1287 non-null   object 
 7   updated_at                       1287 non-null   object 
 8   sbu                              1287 non-null   object 
 9   item_id                          1287 non-null   object 
 10  sub_cluster                      1287 non-null   object 
 11  part_number_clean                213 non-null    object 
 12  nama_item_clean     

In [47]:
df=df.drop([ 'S1',
 'S2',
 'S3',
 'S4',
 'S5',
 'S6',
 'S7',
 'S8',
 'S9',
 'S10'],axis=1)

In [28]:
# df['nama_item_new'] = df['nama_item'].copy()
h = ['SOK / BUBUT RUMAH BEARING','SOK BEARING RUMAH','SOK RUMAH BEARING 6208','SOK RUMAH BEARING 6208','BEARING 18010 SOK']
target= 'SOK BUSHING RUMAH BEARING AS GEAR DRILLING MANUAL'
for target in h:
    print(res)

100
100
100
100
100


In [55]:
# df['ref_nama_item']=df['ref_nama_item'].fillna(0,inplace=True)

In [81]:
# df['nama_item_new'] = df['nama_item_new'].str.replace('2Z','ZZ')
# Check for missing values and fill them with an empty string
# df['REF.ref_nama_item']=df['REF.ref_nama_item'].fillna('-4')
df['ref_nama_item'] = df['REF.nama_item'].copy()

In [61]:
# df[df['ref_nama_item'].str.contains('TRACKER',case=False)]

## Cleaning by cluster

In [343]:
#45
#229
#514
#345
#283
#108
#86
#119
#65
#150
counts= df['agg_cluster_label_REF.nama_item'] ==160
for idx, row in df.iterrows():
    if counts[idx]:
        print(row['ref_nama_item'])

BEARING 1217 K
BEARING 1213 K


In [341]:
df.loc[counts, 'ref_nama_item'] = df.loc[counts, 'ref_nama_item'].str.replace('UCF208J', 'UCF 208 J').str.strip()
print(df.loc[counts, 'ref_nama_item'])

462    BEARING UCF 208 J
463     BEARING UCFL 208
Name: ref_nama_item, dtype: object


In [288]:
s=df['agg_cluster_label_REF.nama_item'].value_counts()
h = (s).value_counts()
print(h)

1     516
2     170
3      77
4      22
5      11
6       3
13      1
9       1
7       1
Name: agg_cluster_label_REF.nama_item, dtype: int64


In [298]:
# Mengambil semua nilai yang memiliki jumlah kemunculan sebanyak 1 atau 2
desired_values = h[(h == 516) | (h == 170)]

# Menampilkan nilai-nilai tersebut
print(desired_values)

1    516
2    170
Name: agg_cluster_label_REF.nama_item, dtype: int64


## Remove Pola

In [10]:
# remove pola W.230-107 
def hapus_pola(teks):
    pola = r'W\.\d{3}-\d{3}'  # Pola yang ingin dihapus
    teks_baru = re.sub(pola, '', teks)
    return teks_baru
df['nama_item_new']=df['nama_item_new'].apply(hapus_pola)

In [3]:
# def remove_dash_at_end(sentence):
#     # Menggunakan pola regex untuk mencocokkan tanda "-" pada akhir kalimat
#     pattern = r'(.+)-$'
#     replacement = r'\1'
#     modified_sentence = re.sub(pattern, replacement, sentence)

#     return modified_sentence

In [4]:
# def sorting_word(kata):
#     if not isinstance(kata, str):
#         kata = str(kata)
    
#     kata_list = kata.split()
    
#     if 'BALL BEARING' in kata_list:
#         kata_list.remove('BALL BEARING')
#         kata_list.insert(0, 'BALL BEARING')
#     elif 'XBEARING' in kata_list:
#         kata_list.remove('XBEARING')
#         kata_list.insert(0, 'XBEARING')
#     else:
#         for i, word in enumerate(kata_list):
#             if word.replace(',', '').isalnum() and not word.replace(',', '').isalpha():
#                 kata_list.remove(word)
#                 kata_list.append(word)
#                 break
    
#     if 'ART' in kata_list and 'BEARING' in kata_list:
#         kata_list.remove('ART')
#         kata_list.remove('BEARING')
#         kata_list.insert(0, 'BEARING')
#         kata_list.insert(1, 'ART')
#     elif 'NTN' in kata_list and 'BEARING' in kata_list:
#         kata_list.remove('NTN')
#         kata_list.remove('BEARING')
#         kata_list.insert(0, 'BEARING')
#         kata_list.insert(1, 'NTN')
#     elif 'NU' in kata_list and 'BEARING' in kata_list:
#         kata_list.remove('NU')
#         kata_list.remove('BEARING')
#         kata_list.insert(0, 'BEARING')
#         kata_list.insert(1, 'NU')
#     elif 'NUTR' in kata_list and 'BEARING' in kata_list:
#         kata_list.remove('NUTR')
#         kata_list.remove('BEARING')
#         kata_list.insert(0, 'BEARING')
#         kata_list.insert(1, 'NUTR')
#     elif 'UCP' in kata_list and 'BEARING' in kata_list:
#         kata_list.remove('UCP')
#         kata_list.remove('BEARING')
#         kata_list.insert(0, 'BEARING')
#         kata_list.insert(1, 'UCP')
#     elif 'FAG' in kata_list and 'BEARING' in kata_list:
#         kata_list.remove('FAG')
#         kata_list.remove('BEARING')
#         kata_list.insert(0, 'BEARING')
#         kata_list.insert(1, 'FAG')
    
    
    
#     numbers = [word for word in kata_list if word.isnumeric()]
#     non_numbers = [word for word in kata_list if not word.isnumeric()]
#     kata_list = non_numbers + numbers
    
#     hyphen_words = [word for word in kata_list if '-' in word]
#     non_hyphen_words = [word for word in kata_list if '-' not in word]
#     manipulated_kata = non_hyphen_words + hyphen_words
    
#     return ' '.join(manipulated_kata).replace(',', ' ')
# df2['kata'] =  df2['nama_item'].apply(sorting_word)
# df2['kata'] =  df2['kata'].apply(remove_dash_at_end)

# #pola untuk BEARING HR.30210-JY yang dipisahkan . atau spasi
# def format_sentence12(sentence):
#     if not isinstance(sentence, str):
#         sentence = str(sentence) 
#     # Menggunakan ekspresi reguler untuk mencari pola HR diikuti dengan angka
#     match = re.search(r'(HR)[.-]?(\d+)[.-:](\w)?', sentence)

#     if match:
#         # Menggabungkan pola HR dan angka dengan spasi di antara keduanya
#         formatted_sentence = re.sub(r'(HR)[.-]?(\d+)[.-:](\w)?', r'\1 \2 \3', sentence)
#         return formatted_sentence

#     # Jika pola tidak sesuai, mengembalikan kalimat asli
#     return sentence

# df['nama_item_new'] = df['kata'].apply(format_sentence12)
# # input_sentence = "BEARING HR.30210:JY"
# # result = format_sentence12(input_sentence)
# # print(result)

# def rearrange_sentence4(sentence):
#     # Menggunakan pola regex untuk menemukan pola "X <angka>" dan menggantinya dengan "<angka> X"
#     pattern = r'\bX\b\s+(\d+)'
#     replacement = r'\1 X'
#     modified_sentence = re.sub(pattern, replacement, sentence)

#     return modified_sentence

# df2['kata'] = df2['kata'].apply(rearrange_sentence4)

# Contoh penggunaan
# sentence = "BEARING TLA X 28 X 35 25"
# hasil = rearrange_sentence4(sentence)
# print(hasil)  # Output: BEARING TLA 28 X 35 X 25


## Remove Word

In [344]:
df.to_excel('C:/Users/Corporate/Documents/ovi/RAW DATA TERBARU/bearing_100323_v2.xlsx')

## Replace WORD

In [11]:
#pola untuk mencari penulisan needle yang salah dengan tingkat kemiripan 80
def find_and_replace_needle(string):
    needle = "NEEDLE"
    similarity_threshold = 80

    words = string.split()  # Membagi string menjadi kata-kata

    for i, word in enumerate(words):
        if fuzz.ratio(word, needle) > similarity_threshold:
            words[i] = needle

    return ' '.join(words)  # Menggabungkan kata-kata menjadi string kembali
df['nama_item_new'] = df['nama_item_new'].apply(find_and_replace_needle)

#pola untuk mencari penulisan thrust yang salah dengan tingkat kemiripan 80
def find_and_replace_thrust(string):
    needle = "THRUST"
    similarity_threshold = 80

    words = string.split()  # Membagi string menjadi kata-kata

    for i, word in enumerate(words):
        if fuzz.ratio(word, needle) > similarity_threshold:
            words[i] = needle

    return ' '.join(words)  # Menggabungkan kata-kata menjadi string kembali
df['nama_item_new'] = df['nama_item_new'].apply(find_and_replace_thrust)

In [54]:
#pola untuk mencari penulisan sock yang salah dengan tingkat kemiripan 80
def find_and_replace_sock(string):
    needle = "SOK"
    similarity_threshold = 70

    words = string.split()  # Membagi string menjadi kata-kata

    for i, word in enumerate(words):
        if fuzz.ratio(word, needle) > similarity_threshold:
            words[i] = needle

    return ' '.join(words)  # Menggabungkan kata-kata menjadi string kembali
df['nama_item_new'] = df['nama_item_new'].apply(find_and_replace_sock)

In [56]:
#pola untuk mencari penulisan tracker yang salah dengan tingkat kemiripan 80
def find_and_replace_tracker(string):
    needle = "TREKER"
    similarity_threshold = 70

    words = string.split()  # Membagi string menjadi kata-kata

    for i, word in enumerate(words):
        if fuzz.ratio(word, needle) > similarity_threshold:
            words[i] = needle

    return ' '.join(words)  # Menggabungkan kata-kata menjadi string kembali
df['nama_item_new'] = df['nama_item_new'].apply(find_and_replace_tracker)

## Move Needle or Thrust Infront

In [29]:
# #pola angka sekitar x
# def format_sentence13(sentence):
#     # Menggunakan ekspresi reguler untuk mencari pola dengan angka di sekitar 'X'
#     match = re.search(r'(\bX\b)\s+(\d+)\s+(\d+)', sentence)

#     if match:
#         # Menggabungkan pola dengan angka dalam urutan yang diinginkan
#         formatted_sentence = re.sub(r'(\bX\b)\s+(\d+)\s+(\d+)', r'\2 \1 \3', sentence)
#         return formatted_sentence

#     # Jika pola tidak sesuai, mengembalikan kalimat asli
#     return sentence
# f ='BEARING TLA X X 28 35 25'
# d = format_sentence13(f)
# print(d)

# df2['kata'] = df2['kata'].apply(format_sentence13)

#kata needle mirip untuk memindahkan ke depan
def move_needle_to_front(row):
    choices = row.split()
    needle_match = process.extractOne("NEEDLE", choices, scorer=fuzz.ratio)
    if needle_match and needle_match[1] > 70:
        choices.remove(needle_match[0])
        choices.insert(0, needle_match[0])
        return ' '.join(choices)
    else:
        return row
df['nama_item_new'] = df['nama_item_new'].apply(move_needle_to_front)

def move_thrust_to_front(row):
    choices = row.split()
    needle_match = process.extractOne("THRUST", choices, scorer=fuzz.ratio)
    if needle_match and needle_match[1] > 70:
        choices.remove(needle_match[0])
        choices.insert(0, needle_match[0])
        return ' '.join(choices)
    else:
        return row
df['nama_item_new'] = df['nama_item_new'].apply(move_thrust_to_front)


#pola untuk move rail
# def move_needle_to_front_RAIL(row):
#     choices = row.split()
#     needle_match = process.extractOne("RAIL", choices, scorer=fuzz.ratio)
#     if needle_match and needle_match[1] > 70:
#         choices.remove(needle_match[0])
#         choices.insert(0, needle_match[0])
#         return ' '.join(choices)
#     else:
#         return row
# df2['kata'] = df2['kata'].apply(move_needle_to_front_RAIL)

# def remove_character_after_4_digits(sentence):
#     # Menggunakan pola regex untuk mencocokkan 4 digit angka diikuti oleh karakter yang akan dihapus
#     pattern = r'(\d{4})(?:-|\.)(\w+)'
#     replacement = r'\1 \2'
#     modified_sentence = re.sub(pattern, replacement, sentence)

#     return modified_sentence
# df2['kata'] = df2['kata'].apply(remove_character_after_4_digits)

# input_sentence = "BEARING 222098-6KK"
# result = remove_character_after_4_digits(input_sentence)
# print(result)

## Extract Bearing Number

In [48]:
df=df.drop('Numbers',axis=1)

In [49]:
# Define a function to extract numbers using regular expressions
def extract_numbers(text):
    numbers = re.findall(r'\d+\.\d+|\d+', text)  # This regex pattern captures integers and decimals
    if numbers:
        return numbers[0]
    else:
        return None
    
# Apply the function to create a new 'Numbers' column
df['Numbers'] = df['nama_item_new'].apply(extract_numbers)

In [6]:
# def separate_ucp(kata):
#     kata = kata.upper()
#     pattern = r'(\bUCP)(\d+)(\w)'
#     replacement = r'\1 \2 \3'
#     kata = re.sub(pattern, replacement, kata)
    
#     return kata

# df2['kata'] = df2['kata'].apply(separate_ucp)
# r = 'BEARING FYH UCP206J'
# d = separate_ucp(r)
# print(d) 

In [61]:
df.to_excel('C:/Users/Corporate/Documents/ovi/item_bearing/edit_bearing/bearing_060923_v3.xlsx',index=False)

In [7]:
def move_words_to_end1(teks):
    if isinstance(teks, list):
        teks = ' '.join(teks)
        
    kata_split = teks.split()

    words_to_move = ["ZZ", "Z","E","NR","J2","2RS","2RS1","RS","BK","K","J","UU","R","A3S","C3","C","D","B","J","EKTN","EX","CAK/C3W33","CAW33C3","CCK/W33","CAK/C4W33","CC/W33","ZZ/C3","J2/Q","/Q",
                    "Z1","DW","2RSR","2RSI","U298","ZZNR","ES","V","TF","2Z/C3","2RSI/C3","C3Z","BEP","ECP","/C3","UZ","LUU","LLU","J7"]

    for word in words_to_move:
        if word in kata_split:
            index_word = kata_split.index(word)
            kata_split.append(kata_split.pop(index_word))

    return ' '.join(kata_split)
f = 'BEARING J 30216'
w = move_words_to_end1(f)
print(w)

df2['kata'] = df2['kata'].apply(move_words_to_end1)

BEARING 30216 J


In [8]:
# df2.to_excel('bearing45.xlsx',index=False)

In [ ]:
def move_words_to_end(teks):
    kata_split = teks.split()

    words_to_move = ["ZZ", "NR"]

    for word in words_to_move:
        if word in kata_split:
            index_word = kata_split.index(word)
            kata_split.append(kata_split.pop(index_word))

    return ' '.join(kata_split)

In [122]:
data.to_excel('bearing_180923_v2.xlsx',index=False)

In [82]:
data = pd.read_excel('bearing_180923_v1.xlsx')
data.head()

,kode_item,nama_item,part_number,jenis_item,kategori_item,status,updated_at,sbu,item_id,sub_cluster,...,_rownum,_processed_at,ref_id,REF.nama_item,REF.part_number,REF.coarse_spec,REF.jenis_item,REF.kategori_item,REF.status,_score
0,SPT.P2.NS.00004510,BEARING SKF 6010 2Z,BEARING SKF 6010 2Z,Stock,SPAREPART,Active,2023-05-24 04:07:28,IPG,IPG-SPT.P2.NS.00004510,BEARING,...,1231,2023-09-11 05:56:34.345551+00:00,IT-629,BEARING SKF 6010 ZZ,NaN,6010,Stock,SPAREPART,Active,100.0
1,MS01175,"BEARING ,30218 J2",NaN,Stock,SPAREPART,Active,2015-01-17 10:17:12,ISP,ISP-MS01175,BEARING,...,377,2023-09-11 05:56:29.674547+00:00,IT-630,BEARING 30218 J2,NaN,30218 J2,Stock,SPAREPART,Active,100.0
2,610504000106,Bearing Type 6307,NaN,Stock,SPAREPART,Active,2023-04-13 09:21:24,IRP,IRP-610504000106,BEARING,...,112,2023-09-11 05:56:29.674547+00:00,IT-631,BEARING 6307,NaN,6307,Stock,SPAREPART,Active,100.0
3,MS01055,BEARING 6307,NaN,Stock,SPAREPART,Active,2019-10-22 08:01:07,ISP,ISP-MS01055,BEARING,...,273,2023-09-11 05:56:29.674547+00:00,IT-631,BEARING 6307,NaN,6307,Stock,SPAREPART,Active,100.0
4,GAS.GA.LL.00000108,BEARING 6307,NaN,Stock,GENERAL SUPPLIES,Active,2020-09-09 02:33:46,IPG,IPG-GAS.GA.LL.00000108,BEARING,...,680,2023-09-11 05:56:29.674547+00:00,IT-631,BEARING 6307,NaN,6307,Stock,SPAREPART,Active,100.0


In [83]:
data['REF.nama_item']=data['REF.nama_item'].fillna(data['nama_item_clean'])

In [84]:
#remove merk 
array_list=['NTN','FAG','SKF','KOYO','THK','NSK','NACHI','FYH','TIMKEN','INA',]

for target in array_list:
    data['REF.nama_item'] = data['REF.nama_item'].str.replace(r'\b' + target, ' ',regex=True)

In [111]:
data['REF.nama_item'] = data['REF.nama_item'].str.replace(r'\b ART \b', ' ',regex=True)

In [86]:
data['REF.nama_item'] = data['REF.nama_item'].str.replace(r'\b IKOBA \b', '',regex=True)
data['REF.nama_item'] = data['REF.nama_item'].str.replace(r'\b IKO \b', '',regex=True)

In [121]:
for i, w in data.iterrows():
    if 'INA' in w['nama_item']:
        print(f"{w['nama_item']} || {w['REF.nama_item']}")

BEARING            ,INA NKX 30 Z || BEARING   NKX 30 Z
BEARING KH 4060    ,INA || BEARING KH 4060  
BEARING KH 2540    ,INA || BEARING KH 2540  
DINAMO  MOTOR 1 HP-GANTI KAPASITOR, BEARING 6203 || DINAMO MOTOR 1 HP-GANTI KAPASITOR BEARING 6203
DINAMO  MOTOR 1 HP-GANTI KAPASITOR,BEARING 6202 || DINAMO MOTOR 1 HP-GANTI KAPASITOR BEARING 6202
INA.NATV.17PPO2/410BEARING ||   NATV 17PPO2/410BEARING
NEEDLE ROLLER      ,KH-5070 INA BEARING || NEEDLE ROLLER KH 5070   BEARING


In [104]:
data['REF.nama_item'] = data['REF.nama_item'].str.replace('BEARING1209', 'BEARING 1209')

In [106]:
data['REF.nama_item'] = data['REF.nama_item'].str.replace('BEARINGNAF 709030', 'BEARING NAF 709030')

In [108]:
data['REF.nama_item'] = data['REF.nama_item'].str.replace('ROLLER BEARING RNAFW 709060IKO', 'ROLLER BEARING RNAFW 709060')

In [113]:
data['REF.nama_item'] = data['REF.nama_item'].str.replace('BEARING121283-4/059', 'BEARING 121283-4/059')